In [2]:
import cv2 as cv
import face_recognition

In [3]:
face_classifier = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

def load_image(image_path):
    image = cv.imread(image_path)
    return image
  
def compute_face_encodings(image):
    face_locations = face_recognition.face_locations(image, model="cnn")
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

def calculate_similarity(known_face_encoding, unknown_face_encoding):
    # Calculate the Euclidean distance between the two face encodings
    return face_recognition.face_distance([known_face_encoding], unknown_face_encoding)

  
def similarity_to_percentage(similarity_score):
  return (1 - similarity_score)* 100

def face_extractor(img) :
  # Function detects face and returns the cropped face
  # if no face is found, it returns the input image
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0 :
        return None
    #crop all faces found
    for (x,y,w,h) in faces :
        cropped_face = img[y : y + h, x : x + w]
    return cropped_face

In [4]:
def main():
    known_image_path = "Known_Faces/Tolu2.jpg"
    known = cv.imread(known_image_path)
    
    cap = cv.VideoCapture(0)
  
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame.")
            break
        
        if face_extractor(frame) is not None :
            face = cv.resize(face_extractor(frame), (200, 200))
            face = cv.cvtColor(face, cv.COLOR_BGR2GRAY)
            unknown_image_path = f"Unknown_Faces/face.jpg"
            cv.imwrite(unknown_image_path, face) 
        else:
            print("\n Face not found \n")
            break
        

        # Load known and unknown face images
        known_image = load_image(known_image_path)
        unknown_image = load_image(unknown_image_path)

        # Compute face encodings for known and unknown faces
        known_face_encodings = compute_face_encodings(known_image)
        unknown_face_encodings = compute_face_encodings(unknown_image)

        if len(known_face_encodings) == 0 or len(unknown_face_encodings) == 0:
            print("No face detected in one or both images.")
            return

        # Calculate similarity score using the face encodings
        similarity_score = calculate_similarity(known_face_encodings[0], unknown_face_encodings[0])
        # print("Similarity Score:", similarity_score[0])
        
        similarity_percentage = float(similarity_to_percentage(similarity_score= similarity_score))
        print(f"Similarity percentage: {similarity_percentage:.2f}%")

        # Define a threshold to consider faces similar or not
        similarity_threshold = 60
        if similarity_percentage >= similarity_threshold:
            print("The faces are similar.")
        else:
            print("The faces are not similar.")
            
        cv.imshow("Known_Faces", known)
        cv.putText(frame, f"Similarity Percentage: {similarity_percentage:.2f}%", (50,50), cv.FONT_HERSHEY_COMPLEX, 1.0, (255,0,255), 2)
        cv.imshow("Unknown Face", frame)
        break
      
    cv.waitKey(0)
    cap.release()
    cv.destroyAllWindows() 

In [8]:
if __name__ == "__main__":
  main()

Similarity percentage: 34.28%
The faces are not similar.
